# Formulário avançado do corredor

Este notebook coleta informações detalhadas do atleta em um formato compatível com o `UserProfile` e com as rotinas de geração de plano (`PlanGenerator`). O fluxo segue recomendações de treinadores como Jack Daniels para entender disponibilidade, experiência, objetivos e restrições antes de gerar o plano.

1. Preencha os widgets com os dados do corredor.
2. Clique em **Gerar perfil + JSON** para visualizar o schema salvo.
3. Use **Gerar plano** para criar um plano com os dados coletados (usa `PlanGenerator` e `TrainingZones`).



In [ ]:

import json
from datetime import date, datetime

import ipywidgets as widgets
from IPython.display import display, clear_output

from user_profile import UserProfile, RaceGoal
from training_zones import TrainingZones
from plan_generator import PlanGenerator

print("Widgets prontos. Preencha os campos e gere o perfil quando terminar.")



## Seções do formulário

Os campos abaixo cobrem todos os atributos usados pelo `PlanGenerator`:
- **Dados pessoais e fisiológicos**: sexo, idade, peso, altura, FC máxima/repouso (usados para estimativas de carga).
- **Experiência e volume**: anos correndo, volume atual/médio/pico e aderência (alimentam cálculo de progressão e risco).
- **Disponibilidade**: dias por semana, horas por dia, blocos críticos e preferência para longão (influenciam agenda e duração de treinos).
- **Objetivos e provas**: prova principal e testes (`RaceGoal`) determinam meta, semanas e ritmo.
- **Ritmos/Zonas**: tempos recentes de prova alimentam `TrainingZones` via método Jack Daniels ou Critical Velocity.
- **Preferências de sessão**: intervalados, tempo run, longão, cross training e mistura de zonas.
- **Lesões e restrições**: definem ajustes, limitações de impacto, zonas vermelhas e rotinas de força.
- **Logística de sessão**: aquecimento, desaquecimento e deslocamento para limitar duração total.



In [ ]:

# --- Widgets: dados pessoais ---
name_w = widgets.Text(description="Nome", value="Corredor(a)")
age_w = widgets.BoundedIntText(description="Idade", value=30, min=12, max=90)
weight_w = widgets.FloatText(description="Peso (kg)", value=70.0)
height_w = widgets.FloatText(description="Altura (cm)", value=175.0)
gender_w = widgets.Dropdown(description="Sexo", options=[("Masculino", "M"), ("Feminino", "F"), ("Não especificar", "")])
hr_rest_w = widgets.IntText(description="FC Repouso", value=0)
hr_max_w = widgets.IntText(description="FC Máx", value=0)

# --- Experiência e volume ---
years_w = widgets.FloatSlider(description="Anos correndo", value=2.0, min=0, max=30, step=0.5)
current_km_w = widgets.FloatSlider(description="Km atual/sem", value=35, min=0, max=200, step=5)
avg_km_w = widgets.FloatSlider(description="Média recente (km)", value=40, min=0, max=250, step=5)
peak_km_w = widgets.FloatSlider(description="Pico recente (km)", value=60, min=0, max=300, step=5)
consistent_days_w = widgets.BoundedIntText(description="Dias mantidos", value=4, min=0, max=7)
experience_w = widgets.Dropdown(description="Nível", options=[("Iniciante", "beginner"), ("Intermediário", "intermediate"), ("Avançado", "advanced")], value="intermediate")
tolerated_w = widgets.SelectMultiple(description="Treinos tolerados", options=UserProfile.TOLERATED_WORKOUT_OPTIONS, layout=widgets.Layout(width="400px"))
adherence_w = widgets.BoundedIntText(description="Aderência %", value=85, min=0, max=100)

# --- Disponibilidade ---
days_week_w = widgets.BoundedIntText(description="Dias/sem", value=4, min=1, max=7)
hours_day_w = widgets.FloatSlider(description="Horas/dia", value=1.0, min=0.5, max=4.0, step=0.5)
preferred_time_w = widgets.Dropdown(description="Horário", options=["", "manhã", "tarde", "noite"], value="")
preferred_location_w = widgets.SelectMultiple(description="Locais", options=["rua", "trilha", "pista", "esteira"])
preferred_days_w = widgets.SelectMultiple(description="Dias preferidos", options=PlanGenerator.DAYS_OF_WEEK)
stressful_blocks_w = widgets.Textarea(description="Blocos críticos", placeholder='{"Monday": ["evening"], "Thursday": ["morning"]}')
long_run_days_w = widgets.SelectMultiple(description="Dias para longão", options=PlanGenerator.DAYS_OF_WEEK)
use_alt_weeks_w = widgets.Checkbox(description="Alternar semanas A/B", value=False)
alt_blocks_w = widgets.Textarea(description="Blocos críticos B", placeholder='{"Tuesday": ["night"]}')
alt_long_days_w = widgets.Textarea(description="Longão semana B", placeholder='["Saturday"]')
weekly_schedule_w = widgets.Textarea(description="Grade semanal", placeholder='{"Monday": [{"start": "06:00", "end": "07:15", "max_minutes": 60, "surfaces": ["rua"]}]}')

# --- Objetivos (RaceGoal) ---
main_distance_w = widgets.Dropdown(description="Prova principal", options=["5K", "10K", "Half Marathon", "Marathon"], value="10K")
main_date_w = widgets.DatePicker(description="Data da prova", value=date.today())
main_name_w = widgets.Text(description="Nome da prova", value="Prova alvo")
main_city_w = widgets.Text(description="Local", value="")
main_target_w = widgets.Text(description="Tempo meta", placeholder="00:45:00")

test_races_w = widgets.Textarea(description="Provas teste", placeholder='[ {"distance": "5K", "date": "2024-08-10", "name": "Park Run"} ]')
secondary_obj_w = widgets.SelectMultiple(description="Objetivos 2ários", options=UserProfile.SECONDARY_OBJECTIVES_OPTIONS)

# --- Ritmos e zonas ---
zones_method_w = widgets.Dropdown(description="Método zonas", options=[("Jack Daniels", "jack_daniels"), ("Critical Velocity", "critical_velocity")], value="jack_daniels")
recent_races_w = widgets.Textarea(description="Tempos recentes", placeholder='{"5K": "00:22:30", "10K": "00:48:00"}')
zone_mix_easy_w = widgets.FloatSlider(description="Easy %", value=0.55, min=0.1, max=0.8, step=0.05)
zone_mix_tempo_w = widgets.FloatSlider(description="Tempo %", value=0.25, min=0.05, max=0.7, step=0.05)
zone_mix_interval_w = widgets.FloatSlider(description="Interval %", value=0.20, min=0.05, max=0.6, step=0.05)
vdot_w = widgets.FloatText(description="VDOT (opc)", value=None)

# --- Preferências de sessão ---
intervals_w = widgets.Checkbox(description="Intervalos", value=True)
tempo_w = widgets.Checkbox(description="Tempo run", value=True)
long_run_w = widgets.Checkbox(description="Longão", value=True)
cross_training_w = widgets.Checkbox(description="Cross training", value=False)
variety_w = widgets.Text(description="Variedade", placeholder="alta/baixa")
social_w = widgets.Text(description="Opções sociais", placeholder="grupo, parceiro")
rpe_w = widgets.IntSlider(description="RPE chave", value=7, min=1, max=10)
long_tol_w = widgets.Text(description="Tolerância longas", placeholder="moderada")
routine_fun_w = widgets.Text(description="Rotina vs diversão", placeholder="equilibrado")

# --- Lesões e restrições ---
prev_inj_w = widgets.SelectMultiple(description="Lesões passadas", options=UserProfile.COMMON_INJURIES)
current_inj_w = widgets.SelectMultiple(description="Lesões atuais", options=UserProfile.COMMON_INJURIES)
triggers_w = widgets.Text(description="Gatilhos", placeholder="descidas, frio")
red_zones_w = widgets.Text(description="Zonas vermelhas", placeholder="paralelepípedo, descidas" )
impact_lim_w = widgets.Text(description="Limites impacto", placeholder="evitar descidas" )
strength_w = widgets.Text(description="Rotinas de força", placeholder="mobilidade quadril 2x/sem")

# --- Logística de sessão ---
warmup_w = widgets.BoundedIntText(description="Aquecimento (min)", value=10, min=0, max=60)
cooldown_w = widgets.BoundedIntText(description="Desaquecimento (min)", value=10, min=0, max=60)
commute_w = widgets.BoundedIntText(description="Deslocamento (min)", value=0, min=0, max=90)

form = widgets.Accordion(children=[
    widgets.VBox([name_w, age_w, gender_w, weight_w, height_w, hr_rest_w, hr_max_w]),
    widgets.VBox([years_w, experience_w, current_km_w, avg_km_w, peak_km_w, consistent_days_w, adherence_w, tolerated_w]),
    widgets.VBox([days_week_w, hours_day_w, preferred_time_w, preferred_location_w, preferred_days_w, long_run_days_w, stressful_blocks_w, use_alt_weeks_w, alt_blocks_w, alt_long_days_w, weekly_schedule_w]),
    widgets.VBox([main_distance_w, main_date_w, main_name_w, main_city_w, main_target_w, test_races_w, secondary_obj_w]),
    widgets.VBox([zones_method_w, recent_races_w, zone_mix_easy_w, zone_mix_tempo_w, zone_mix_interval_w, vdot_w]),
    widgets.VBox([intervals_w, tempo_w, long_run_w, cross_training_w, variety_w, social_w, rpe_w, long_tol_w, routine_fun_w]),
    widgets.VBox([prev_inj_w, current_inj_w, triggers_w, red_zones_w, impact_lim_w, strength_w]),
    widgets.VBox([warmup_w, cooldown_w, commute_w])
])
form.set_title(0, "Pessoais")
form.set_title(1, "Experiência")
form.set_title(2, "Disponibilidade")
form.set_title(3, "Objetivos")
form.set_title(4, "Zonas e ritmos")
form.set_title(5, "Preferências de sessão")
form.set_title(6, "Lesões e restrições")
form.set_title(7, "Logística")

display(form)



In [ ]:

def _safe_json(text, default):
    if not text or text.strip() == "":
        return default
    try:
        return json.loads(text)
    except Exception as exc:
        print(f"JSON inválido ({exc}); usando padrão {default}")
        return default

def _split_text(text):
    if not text:
        return []
    return [item.strip() for item in text.split(',') if item.strip()]



In [ ]:

def build_profile_from_widgets():
    main_goal = RaceGoal(
        distance=main_distance_w.value,
        date=main_date_w.value or date.today(),
        name=main_name_w.value,
        location=main_city_w.value,
        is_main_goal=True,
        target_time=main_target_w.value or None,
    )

    test_races = []
    for race_dict in _safe_json(test_races_w.value, []):
        try:
            test_races.append(RaceGoal.from_dict(race_dict))
        except Exception as exc:
            print(f"Ignorando prova teste inválida: {race_dict} ({exc})")

    zone_mix = {
        "easy": float(zone_mix_easy_w.value),
        "tempo": float(zone_mix_tempo_w.value),
        "interval": float(zone_mix_interval_w.value),
    }

    profile = UserProfile(
        name=name_w.value,
        age=int(age_w.value or 0),
        weight_kg=float(weight_w.value or 0),
        height_cm=float(height_w.value or 0),
        gender=gender_w.value,
        hr_resting=int(hr_rest_w.value or 0) or None,
        hr_max=int(hr_max_w.value or 0) or None,
        years_running=float(years_w.value or 0),
        current_weekly_km=float(current_km_w.value or 0),
        average_weekly_km=float(avg_km_w.value or 0),
        recent_peak_weekly_km=float(peak_km_w.value or 0),
        consistent_days_per_week=int(consistent_days_w.value or 0),
        tolerated_workouts=list(tolerated_w.value),
        adherence_score=int(adherence_w.value or 0),
        experience_level=experience_w.value,
        main_race=main_goal,
        test_races=test_races,
        secondary_objectives=list(secondary_obj_w.value),
        days_per_week=int(days_week_w.value),
        hours_per_day=float(hours_day_w.value),
        preferred_time=preferred_time_w.value,
        preferred_location=list(preferred_location_w.value),
        preferred_days=list(preferred_days_w.value),
        stressful_blocks=_safe_json(stressful_blocks_w.value, {}),
        long_run_preference_days=list(long_run_days_w.value),
        use_alternating_weeks=bool(use_alt_weeks_w.value),
        alternate_stressful_blocks=_safe_json(alt_blocks_w.value, {}),
        alternate_long_run_days=_safe_json(alt_long_days_w.value, []),
        weekly_schedule=_safe_json(weekly_schedule_w.value, {}),
        default_warmup_minutes=int(warmup_w.value or 0),
        default_cooldown_minutes=int(cooldown_w.value or 0),
        commute_minutes=int(commute_w.value or 0),
        typical_key_workout_rpe=int(rpe_w.value or 0) if rpe_w.value else None,
        long_session_tolerance=long_tol_w.value,
        variety_preference=variety_w.value,
        social_training_options=_split_text(social_w.value),
        routine_vs_fun_balance=routine_fun_w.value,
        recent_race_times=_safe_json(recent_races_w.value, {}),
        zones_calculation_method=zones_method_w.value,
        zone_mix_preference=zone_mix,
        vdot_estimate=float(vdot_w.value) if vdot_w.value else None,
        session_preferences={
            "intervals": intervals_w.value,
            "tempo": tempo_w.value,
            "long_run": long_run_w.value,
            "cross_training": cross_training_w.value,
        },
        previous_injuries=list(prev_inj_w.value),
        current_injuries=list(current_inj_w.value),
        injury_triggers=_split_text(triggers_w.value),
        red_zones=_split_text(red_zones_w.value),
        impact_limitations=_split_text(impact_lim_w.value),
        strength_routines=_split_text(strength_w.value),
    )

    return profile


def on_generate_profile(_=None):
    clear_output(wait=True)
    display(form, generate_btn, save_json_btn, plan_btn)
    profile = build_profile_from_widgets()

    # Calcular zonas (se informado)
    zones = None
    if profile.recent_race_times:
        zones = TrainingZones(method=profile.zones_calculation_method)
        for dist, tempo in profile.recent_race_times.items():
            zones.add_race_result(distance_km=TrainingZones._distance_from_label(dist), time_str=tempo)

    profile_dict = profile.to_dict()
    print("
Resumo do perfil gerado:")
    print(profile)
    print("
Generator params:", profile.to_generator_params())
    if zones:
        print("
Zonas calculadas:")
        print(zones)

    # Guardar JSON em memória para exportar
    on_generate_profile.profile_cache = profile_dict


def on_save_json(_=None):
    profile_dict = getattr(on_generate_profile, "profile_cache", None)
    if not profile_dict:
        on_generate_profile()
        profile_dict = getattr(on_generate_profile, "profile_cache", None)
    if not profile_dict:
        print("Nenhum perfil disponível para salvar.")
        return
    filename = f"perfil_corredor_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(profile_dict, f, indent=2, ensure_ascii=False)
    print(f"Perfil salvo em {filename}")


def on_generate_plan(_=None):
    clear_output(wait=True)
    display(form, generate_btn, save_json_btn, plan_btn)
    profile = build_profile_from_widgets()
    zones = None
    if profile.recent_race_times:
        zones = TrainingZones(method=profile.zones_calculation_method)
        for dist, tempo in profile.recent_race_times.items():
            zones.add_race_result(distance_km=TrainingZones._distance_from_label(dist), time_str=tempo)

    plan = PlanGenerator.generate_plan(
        name=f"Plano {profile.main_race.distance}",
        goal=profile.main_race.distance,
        level=profile.experience_level,
        days_per_week=profile.days_per_week,
        training_zones=zones,
        user_profile=profile,
    )
    print(plan)

# Botões de ação
generate_btn = widgets.Button(description="Gerar perfil + JSON", button_style="primary")
save_json_btn = widgets.Button(description="Salvar JSON", button_style="success")
plan_btn = widgets.Button(description="Gerar plano", button_style="info")


generate_btn.on_click(on_generate_profile)
save_json_btn.on_click(on_save_json)
plan_btn.on_click(on_generate_plan)

display(widgets.HBox([generate_btn, save_json_btn, plan_btn]))

